# PSF Photometry using effective PSF models (in python!)

We present a software stack for fitting the PSF models developed by J Anderson for HST/WFC3 to perform high precision PSF photometry.  In general, the process is as follows:
1. Find and fit stars in each image using the PSF model
2. Align the images using the PSF fitted catalogs
3. Match the stars and collate/average the measurements into one final catalog.
In this notebook, we show a simple workflow for performing these tasks using the psf_tools package.



### Versions!

NOTE: These tools require an installation of astroconda, using python 3.6, and photutils >= 0.7.1.  It also requires astroquery and skimage >=0.15.0

# Table of Contents
0. [Data Download](#dl)

1a. [Fortran Interface](#fortran)

1b. [Python Interface](#python)


2. [Align Images](#align)


3. [Collate/Average Measurements](#average)


4. [Results](#results)

In [1]:
# some parameters we will use consistently
hmin = 5
fmin = 1000.
pmax= 66000.

In [2]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os

from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Observations

from psf_tools import run_hst1pass, run_python_psf_fitting, align_images, make_final_table, \
                    match_final_catalogs, match_to_master_catalog



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


In [3]:
import matplotlib as mpl
mpl.rcParams['xtick.labelsize'] = 10
plt.rcParams.update({'axes.titlesize' : '18',
                     'axes.labelsize' : '14',
                     'xtick.labelsize' : '14',
                     'ytick.labelsize' : '14'})
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
%matplotlib inline

# 0. First, download some data
<a id="dl">

Let's download some data from the outer regions of Omega Cen to start.  We will use images in 2 filters, 2 epochs each.

In [4]:
cd oc/

/grp/hst/wfc3i/irivera/repos/wfc3_photometry/oc


In [5]:
obsTable = Observations.query_criteria(project='HST',proposal_id='14118', obs_id='ICTJ4[5678]*', obstype='all')
products = Observations.get_product_list(obsTable)
filtered_products = Observations.filter_products(products,mrp_only=False, productSubGroupDescription='FLC')
dl_tbl = Observations.download_products(filtered_products,mrp_only=False)
files = []
for f in dl_tbl['Local Path']:
    filename = os.path.split(f)[-1]
    files.append(filename)
    if not os.path.exists(filename):
        shutil.move(f, '.')


INFO: Found cached file ./mastDownload/HST/hst_14118_45_wfc3_uvis_f814w_ictj45ah/hst_14118_45_wfc3_uvis_f814w_ictj45ah_flc.fits with expected size 169148160. [astroquery.query]
INFO: Found cached file ./mastDownload/HST/hst_14118_45_wfc3_uvis_f814w_ictj45aj/hst_14118_45_wfc3_uvis_f814w_ictj45aj_flc.fits with expected size 169263360. [astroquery.query]
INFO: Found cached file ./mastDownload/HST/hst_14118_45_wfc3_uvis_f814w_ictj45b9/hst_14118_45_wfc3_uvis_f814w_ictj45b9_flc.fits with expected size 169263360. [astroquery.query]
INFO: Found cached file ./mastDownload/HST/hst_14118_46_wfc3_uvis_f814w_ictj46cs/hst_14118_46_wfc3_uvis_f814w_ictj46cs_flc.fits with expected size 169044480. [astroquery.query]
INFO: Found cached file ./mastDownload/HST/hst_14118_46_wfc3_uvis_f814w_ictj46cu/hst_14118_46_wfc3_uvis_f814w_ictj46cu_flc.fits with expected size 169044480. [astroquery.query]
INFO: Found cached file ./mastDownload/HST/hst_14118_46_wfc3_uvis_f814w_ictj46cz/hst_14118_46_wfc3_uvis_f814w_ictj4

### Multiple filters in downloaded data, only look at/fit one band at a time

In [6]:
files = glob.glob('ictj*flc.fits')

In [7]:
files

['ictj47v1q_flc.fits',
 'ictj48dgq_flc.fits',
 'ictj45ahq_flc.fits',
 'ictj46cuq_flc.fits',
 'ictj45ajq_flc.fits',
 'ictj48d7q_flc.fits',
 'ictj45b9q_flc.fits',
 'ictj46czq_flc.fits',
 'ictj47v4q_flc.fits',
 'ictj48d9q_flc.fits',
 'ictj46csq_flc.fits',
 'ictj47uzq_flc.fits']

In [8]:
input_images =[im for im in  sorted(files) if fits.getval(im, 'FILTER')== 'F814W']

In [9]:
input_images

['ictj45ahq_flc.fits',
 'ictj45ajq_flc.fits',
 'ictj45b9q_flc.fits',
 'ictj46csq_flc.fits',
 'ictj46cuq_flc.fits',
 'ictj46czq_flc.fits']

### Lets fit just the longer exposures in this example

In [10]:
input_images = [im for im in input_images if fits.getval(im, 'EXPTIME') > 200.]

In [11]:
for im in input_images:
    print(im, fits.getval(im, 'exptime'), fits.getval(im, 'date-obs'), fits.getval(im, 'filter'))

ictj45ajq_flc.fits 1253.0 2016-06-27 F814W
ictj45b9q_flc.fits 1345.0 2016-06-27 F814W
ictj46cuq_flc.fits 1253.0 2016-07-04 F814W
ictj46czq_flc.fits 1345.0 2016-07-04 F814W


# 1a. Fortran Interface<a id="fortran"></a>

### See input options:

In [12]:
run_hst1pass?

Signature:
run_hst1pass(
    input_images,
    hmin=5,
    fmin=1000,
    pmax=99999,
    out='xympqks',
    executable_path=None,
    **kwargs,
)
Docstring:
Run hst1pass.e Fortran code on images to produce initial catalogs.

This function runs the Fortran routine hst1pass.e, on a set of
input images.  This is Jay Anderson's PSF fitting single pass
photometry code.  The fortran code must first be compiled for
this code to run.  Running this code outputs one catalog for
each input image, and saves those files to disk.  More parameters
from the hst1pass.e code may be added to this interface in the
future.  The original Fortran executable can also be called from
the command line if desired.

Parameters
----------
input_images : list
    List of image filenames (strings).  Must be fits files.
hmin : int
    Minimum separation between stars. Default 5
fmin : int, optional
    The minimum flux (in image units) a source must have to be
    included in the output catalogs. Default 1000.
pmax :

In general, the default parameters passed are suitable for most use cases.  The only parameters that would be recommeneded to edit would be `fmin`, `pmax`, and `hmin`.  There are also extra keyword arguments that can be passed to the call, but support for those parameters is not guaranteed.

To see other keyword arguments, run the fortran executable with no arguments

In [13]:
import psf_tools
print(psf_tools.PSFPhot._get_exec_path())

/grp/hst/wfc3i/irivera/repos/wfc3_photometry/psf_tools/hst1pass_linux.e


Replace the path below with whatever was output above

In [14]:
! /Users/vbajaj/Documents/wfc3_photometry/wfc3_photometry/psf_tools/hst1pass_darwin.e

/bin/bash: /Users/vbajaj/Documents/wfc3_photometry/wfc3_photometry/psf_tools/hst1pass_darwin.e: No such file or directory


### Run the PSF photometry

The executable is ran in the following cell using the Fortran engine.  Supplying `focus=-1` parameter tells the software to determine and use the focus dependent PSF library.  If not supplied then the standard (spatially dependent, but non focus dependent PSFs are used.

In [15]:
catalogs = run_hst1pass(input_images=input_images, hmin=hmin, fmin=fmin, pmax=pmax)

Using PSF file STDPSF_WFC3UV_F814W.fits
/grp/hst/wfc3i/irivera/repos/wfc3_photometry/psf_tools/hst1pass_linux.e HMIN=5 FMIN=1000.0 PMAX=66000.0 OUT=xympqks GDC=NONE PSF=STDPSF_WFC3UV_F814W.fits ictj45ajq_flc.fits ictj45b9q_flc.fits ictj46cuq_flc.fits ictj46czq_flc.fits
b''
b''
b'ARG0000  /grp/hst/wfc3i/irivera/repos/wfc3_photometry/psf_tools/hst1pass_linux.e'
b'ARG0001  HMIN=5'
b'ARG0002  FMIN=1000.0'
b'ARG0003  PMAX=66000.0'
b'ARG0004  OUT=xympqks'
b'--->    NLISTs =            1'
b'--->    NITEMs =            7'
b'--->   OUTLIST = xympqks'
b'ARG0005  GDC=NONE'
b'ARG0006  PSF=STDPSF_WFC3UV_F814W.fits'
b'ARG0007  ictj45ajq_flc.fits'
b'---> NIM0001 ictj45ajq_flc.fits'
b'ARG0008  ictj45b9q_flc.fits'
b'---> NIM0002 ictj45b9q_flc.fits'
b'ARG0009  ictj46cuq_flc.fits'
b'---> NIM0003 ictj46cuq_flc.fits'
b'ARG0010  ictj46czq_flc.fits'
b'---> NIM0004 ictj46czq_flc.fits'
b''
b''
b''
b''
b'OUTPUT FROM PROGRAM hst2xym'
b''
b'/grp/hst/wfc3i/irivera/repos/wfc3_photometry/psf_tools/hst1pass_linux.e'


### See the output catalogs

In [16]:
# If you already have the output catalogs, and dont want to rerun hst1pass
catalogs = [im.replace('.fits', '.xympqks') for im in input_images]

In [17]:
catalogs

['ictj45ajq_flc.xympqks',
 'ictj45b9q_flc.xympqks',
 'ictj46cuq_flc.xympqks',
 'ictj46czq_flc.xympqks']

'If ran with the flag `focus = -1` (uses focus dependent PSF) focus of image can be output

In [18]:
from psf_tools.PSFPhot import check_focus

In [19]:
check_focus(catalogs)

{'ictj45ajq_flc.xympqks': 0.0,
 'ictj45b9q_flc.xympqks': 0.0,
 'ictj46cuq_flc.xympqks': 0.0,
 'ictj46czq_flc.xympqks': 0.0}

In [20]:
output_catalogs = []
for im in input_images:
    cat_str = im.replace('.fits', '_sci?_xyrd.cat')
    output_catalogs += glob.glob(cat_str)
output_catalogs = sorted(output_catalogs)

##### The hst1pass files are broken up per chip.  The following catalogs should be used for the exposure level analysis

In [21]:
Table.read('ictj45ajq_flc_sci1_xyrd.cat', format='ascii.commented_header')

x,y,r,d,m,p,q,s
float64,float64,float64,float64,float64,float64,float64,float64
3161.462,5.393,201.53878582782485,-47.64694528689246,-8.226,465.42,0.816,32.91
3456.623,5.509,201.53576201399736,-47.649542492980686,-12.244,10127.43,0.036,61.39
...,...,...,...,...,...,...,...
2161.211,2040.743,201.52148625332956,-47.625360543141355,-8.792,498.82,0.126,36.64
3060.84,2040.752,201.51229751438623,-47.63316340089551,-9.445,991.29,0.089,31.18
1648.016,2042.175,201.52668624588918,-47.62091762708007,-12.283,13464.14,0.029,54.54


In [22]:
output_catalogs

['ictj45ajq_flc_sci1_xyrd.cat',
 'ictj45ajq_flc_sci2_xyrd.cat',
 'ictj45b9q_flc_sci1_xyrd.cat',
 'ictj45b9q_flc_sci2_xyrd.cat',
 'ictj46cuq_flc_sci1_xyrd.cat',
 'ictj46cuq_flc_sci2_xyrd.cat',
 'ictj46czq_flc_sci1_xyrd.cat',
 'ictj46czq_flc_sci2_xyrd.cat']

# Python interface<a id="python"></a>

A fully python based interface is  available for the PSF fitting.  The python interface has a pared down feature set, and does not yet support the focus based PSFs (coming soon!), and runs slower, but has a more familiar usage compared to the Fortran interface.   The fortran interface can also much more reliably measure UVIS saturated stars, whereas the Python interface cannot. The output `xyrd` catalogs are the same as the fortran, and so either interface can be selected for use for the initial fittings, and then the following alignment/collating steps are the same.

### Look at inputs

In [23]:
run_python_psf_fitting?

Signature:
run_python_psf_fitting(
    input_images,
    psf_model_file=None,
    hmin=5,
    fmin=1000.0,
    pmax=70000.0,
    qmax=0.5,
    cmin=-1.0,
    cmax=0.1,
    ncpu=None,
    focus_only=False,
)
Docstring:
Main function to run to do finding/PSF fitting of stars with python engine

This is the top level function of the python PSF photometry interface.
This gets the PSF and other information, loops over the images and calls
the fitting functions.  The inputs control the star finding/rejection
criteria.  The outputs photometric catalogs are saved as ascii tables.

Note: Focus dependent PSFs not yet supported in the python engine (but
are coming soon)

Parameters
----------
input_images : list
    List of image filenames (strings) to measure.  Must be fits files.
psf_model_file : str, optional
    Name of fits file containing PSF model.  If None (default),
    finds/downloads the appropriate model from WFC3 website.
hmin : int, optional
    Minimum separation (in pixels) betwee

In general, the parameters to the python interface are very similar.  More of the keyword arguments are specified directly (qmax, cmin, cmax), and have docstrings specifying their use.  You can also specify the number of CPUs to distribute the fitting across, which subtrantially improves performance when > 1 (default is to use all CPUs).  

### Run the python fitting

In [24]:
run_python_psf_fitting(input_images, hmin=hmin, fmin=fmin, pmax=pmax)

Using PSF file STDPSF_WFC3UV_F814W.fits
ictj45ajq_flc.fits
Rejected 13608 of 28067 sources for being too peaked


PicklingError: Can't pickle <functools._lru_cache_wrapper object at 0x7fe686a9f3d0>: it's not the same object as photutils.psf.griddedpsfmodel.GriddedPSFModel._calc_interpolator_uncached

### Look at one of the xyrd catalogs

The xyrd catalogs contain the outputs from the fitting.  Depending on what was selected in the fitting parameters, the number of columns may vary.  However, the important columns are:

`x`, `y`: The pixel positions of the source measured (1-indexed)

`r`, `d`: The decimal degree RA and Dec of the source measured

`m`: The instrumental magnitude of the fitted PSF (`-2.5 * np.log10(flux [electrons])`)

`q`: The fit quality (see docstring of `run_python_psf_fitting()` for more info)

In [ ]:
Table.read(output_catalogs[0], format='ascii.commented_header')

# 2. Align the images <a id="align"></a>

With the image based catalogs, we can now align them using the accurate positions from the PSF fitting steps.  The alignment not only removes pointing error between images, but also makes the images/catalogs useful for astrometric studies.  This step also corrects the RAs/Decs in the catalogs with the updated values resulting from the alignment.  Furthermore, aligning the images is crucial for tbe final matching/averaging step.

### See options for aligning the images using the catalogs from previous step

In [ ]:
align_images?

Setting `gaia=True` will automatically download and align the catalogs to Gaia DR2 (no proper motions added due to low precision of Gaia proper motions).  This yields high accuracy absolute astrometric measurements!

In [ ]:
align_images(input_images, searchrad=1.5, gaia=True, wcsname='DEMO', prop=True)


### Can also run with any arguments passed to TweakReg


In [ ]:
align_images(input_images, searchrad=.1, gaia=True, fluxcol=5, maxflux=-14., minflux=-11., fluxunits='mag')

#### Can look at shift file, saved as shifts.txt

The columns here are `xshift`, `yshift` (pixels), `rotation` (deg), `scale`, `xrms`, and `yrms`(pixels)

In [ ]:
cat shifts.txt

Aligning to gaia may result in slightly higher RMS values than aligning relatively (`gaia=False` and  `reference_catalog=None`) due to proper motions/inaccuracies in the gaia catalog.

In [ ]:
align_images(input_images, searchrad=.1, fluxcol=5, maxflux=-14., minflux=-11., fluxunits='mag')

In [ ]:
cat shifts.txt

# 3 Match/Average the catalogs into final catalog<a id="average"></a>

The final step takes the astrometrically corrected images/catalogs, and matches common stars between them, averaging those measurements into one final catalog as well as providing errors on each of those measurements, while clipping erroneous values.  It also flux calibrates (applies aperture, pixel area map, and zerpoint correction) the magnitudes, to make them ready for science!  This software also has the ability to produce a simple drizzled image of the input images, as to have a corresponding data file

### See options for final collation of the table

In [ ]:
make_final_table?

In [ ]:
tbl_i = make_final_table(input_images, min_detections=3)

In [ ]:
tbl_i

# 4. See the results!<a id="results"></a>
The columns are mbar, rbar, dbar, qbar, xbar, and ybar which are mean magnitude (instrumental), RA, Dec, Q (fit quality), X position in the output frame, and Y position in the output frame.  The columns ending in 'std' are the standard deviations of the values.  The column 'n' is the number of times that source was detected (and not clipped out from the averaging).  The column `n_expected` is the number of images that covered that sky position, and thus the total number of times that star could have been detected by the input images.

### To access a column from the table, the syntax is `tbl[<colname>]` as seen below

In [ ]:
tbl_i = Table.read('F814W_final_cat.txt', format='ascii.commented_header')

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(tbl_i['rbar'], tbl_i['dbar'], c=tbl_i['mbar'], s = 8, alpha=.3)
plt.xlim(plt.xlim()[::-1]) # reverse x axis for RA
plt.xlabel('RA')
plt.xlabel('Dec')
cb = plt.colorbar()
cb.set_label('I [mag]')
plt.grid(ls=':')

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(tbl_i['mbar'], tbl_i['mstd'], c=tbl_i['qbar'], s = 8, alpha=.3, vmin=0., vmax=.1, cmap='viridis')
plt.ylim(-.0,.1)
plt.xlabel('I [mag]')
plt.ylabel('$\sigma_I$')
cb = plt.colorbar()
cb.set_label('qbar')
plt.grid(ls=':')

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(tbl_i['xstd'], tbl_i['ystd'], s = 5, alpha=.25, c=tbl_i['mbar'], cmap='viridis')
plt.xlim(0.0001, 1)
plt.ylim(0.0001, 1)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('xstd')
plt.ylabel('ystd')
cb = plt.colorbar()
cb.set_label('mbar [mag]')
plt.grid(ls=':')

In [ ]:
# stat, bedges, bn = binned_statistic(tbl['mbar'], tbl['xstd'],statistic=np.nanmedian, bins=20)
fig = plt.figure(figsize=(10,10))
plt.scatter(tbl_i['mbar'], tbl_i['xstd'], s=6, alpha=.4, c=tbl_i['qbar'], vmin=0, vmax=.2, cmap='viridis')

plt.xlabel('$\overline{m}$')
plt.ylabel('x std')
plt.grid(ls=':')
plt.ylim(-.0,.1)
plt.xlim(17, 27)

### Tables can be matched as well:

In [ ]:
gaia_tbl = Table.read('gaia.cat', format='ascii.commented_header')
gaia_tbl['ra'].name = 'rbar'
gaia_tbl['dec'].name = 'dbar'
matched_i, matched_gaia = match_final_catalogs(tbl_i, gaia_tbl, max_distance=.04)

In [ ]:
plt.scatter(matched_i['mbar'], (matched_i['rbar'] - matched_gaia['rbar'])*3600., s=4, alpha=.4)
plt.xlim(17., 22.)
plt.ylim(-.01, .01)
plt.ylabel('delta RA [arcsec]')
plt.xlabel('mbar')
plt.grid(ls=':')

In [ ]:
plt.scatter(matched_i['mbar'], (matched_i['dbar'] - matched_gaia['dbar'])*3600., s=4, alpha=.4)
plt.xlim(17., 22.)
plt.ylim(-.01, .01)
plt.ylabel('delta DEC [arcsec]')
plt.xlabel('mbar')
plt.grid(ls=':')

### Match tables from two filters

Run the same steps, with the F606W images to get an averaged catalog of those files.  Then we can match the F814W and F606W tables and make a simple CMD

In [ ]:
v_images =  [im for im in files if fits.getval(im, 'FILTER')=='F606W']
v_images =  [im for im in v_images if fits.getval(im, 'EXPTIME')>200]
run_hst1pass(v_images, hmin=hmin, fmin=fmin, pmax=pmax)
align_images(v_images, searchrad=1., gaia=True)
tbl_v = make_final_table(v_images, min_detections=3)

In [ ]:
tbl_v = Table.read('F606W_final_cat.txt', format='ascii.commented_header')

In [ ]:
matched_i, matched_v = match_final_catalogs(tbl_i, tbl_v)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(matched_v['mbar'] - matched_i['mbar'], matched_v['mbar'], s=3, alpha=.3)
plt.ylim(plt.ylim()[::-1])
plt.xlabel('V - I [mag]')
plt.ylabel('V [mag]')
plt.title('Color Magnitude Diagram')
plt.xlim(-1,1)
plt.ylim(28, 17)

# 4. Using models for other analysis
In addition to fitting stars in clusters, there are also tools for using the PSF models for addition/subtraction of stars

In [ ]:
from psf_tools.PyFitting import subtract_psfs, make_models, get_subtrahend
from matplotlib.colors import LogNorm

In [ ]:
data = fits.getdata('ictj45ajq_flc.fits')
mod1, mod2 = make_models('../psf_tools/PSFSTD_WFC3UV_F814W.fits')
cat = Table.read('ictj45ajq_flc_sci1_xyrd.cat', format='ascii.commented_header')
subbed = subtract_psfs(data, cat, mod1)

In [ ]:
plt.figure(figsize=(15,7))
plt.imshow(data, vmin=12, vmax=1100, norm=LogNorm(), origin='lower')
plt.xlim(0, 500)
plt.ylim(0, 500)

In [ ]:
plt.figure(figsize=(15,7))
plt.imshow(subbed, vmin=12, vmax=1100, norm=LogNorm(), origin='lower')
plt.xlim(0, 500)
plt.ylim(0, 500)

In [ ]:
fluxes = np.power(10, cat['m']/-2.5).data
# subtrahend = get_subtrahend(cat['x'], cat['y'], fluxes, mod1, data.shape)
plt.figure(figsize=(15,7))
plt.imshow(np.abs(subtrahend), vmin=.001, vmax=1100, norm=LogNorm(), origin='lower')
plt.xlim(0, 500)
plt.ylim(0, 500)